In [ ]:
import importlib

import librosa
import pandas as pd
import tensorflow_io as tfio
import tensorflow as tf

import src.preprocess
importlib.reload(src.preprocess)

DATA_ROOT = "gs://bird-clef-kimmo/data"

In [ ]:
audio = tfio.audio.AudioIOTensor(f'{DATA_ROOT}/train_short_audio/mallar3/XC104614.ogg')
print(audio)

In [ ]:
audio_tensor = tf.squeeze(audio.to_tensor(), axis=[-1])
audio_tensor = audio_tensor[:32000*5]
print(audio_tensor)

In [ ]:
from IPython.display import Audio

Audio(audio_tensor.numpy(), rate=audio.rate.numpy())

In [ ]:
import matplotlib.pyplot as plt

tensor = tf.cast(audio_tensor, tf.float32) / 32768.0

plt.figure()
plt.plot(tensor.numpy())

In [ ]:
spectrogram = tfio.audio.spectrogram(
    tensor, nfft=512, window=512, stride=256)

plt.figure()
plt.imshow(tf.math.log(spectrogram).numpy())

In [ ]:
spectrogram.shape

In [ ]:
# Convert to mel-spectrogram
mel_spectrogram = tfio.audio.melscale(
    spectrogram, rate=16000, mels=128, fmin=0, fmax=8000)

plt.figure()
plt.imshow(tf.math.log(mel_spectrogram).numpy())

# Convert to db scale mel-spectrogram
dbscale_mel_spectrogram = tfio.audio.dbscale(
    mel_spectrogram, top_db=80)

plt.figure()
plt.imshow(dbscale_mel_spectrogram.numpy())

In [ ]:
freq_mask = tfio.audio.freq_mask(dbscale_mel_spectrogram, param=10)

plt.figure()
plt.imshow(freq_mask.numpy())

In [ ]:
time_mask = tfio.audio.time_mask(dbscale_mel_spectrogram, param=10)

plt.figure()
plt.imshow(time_mask.numpy())